In [1]:
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          Trainer,
                          pipeline,
                          DataCollatorForLanguageModeling,
                          PreTrainedTokenizer)
from peft import LoraConfig, get_peft_model
import huggingface_hub
import os
import logging
from tqdm import tqdm
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from typing import List
import wandb
from lora_llm import evaluate

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [ ]:
!pip install -U bitsandbytes

In [ ]:
!pip install transformers
!pip install peft
!pip install 'accelerate>=0.26.0'
!pip install -U bitsandbytes
!pip install huggingface-hub
!pip install datasets
!pip install wandb

In [3]:
with open('secrets.json', 'r') as file:
    secrets = json.load(file)
    huggingface_hub.login(secrets.get('HF_KEY'))

class_names = []
base_model_name = 'meta-llama/Llama-3.1-8B-Instruct'
label_name = "evasion_label"
fine_tuned_model_path = f"./llama3.1"

evaluate(base_model_name,
         fine_tuned_model_path,
         "evasion_label",
         "clarity_label",
         "preprocessed_data/test_set_unnamed.csv")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
 12%|█▎        | 1/8 [00:04<00:30,  4.42s/it]

Right label: dodging 

### selected
Right label: dodging 

### part
Right label: dodging 

### explicit
Right label: dodging 

### selected
Right label: dodging 

### part
Right label: general 

### selected question
Right label: dodging 

### selected
Right label: explicit 

### explicit label


 25%|██▌       | 2/8 [00:08<00:25,  4.29s/it]

Right label: explicit 

### selected question
Right label: general 

### selected question
Right label: explicit 

### explicit label
Right label: clarification 

### part
Right label: dodging 

### selected
Right label: dodging 

### selected
Right label: clarification 

### selected
Right label: explicit 

### selected question


...

Right label: explicit 

### part of
Right label: explicit 

### selected question
Right label: dodging 

### selected
Right label: explicit 

### part of
Right label: clarification 

### part
Right label: explicit 

### selected question
Right label: explicit 

### part of
Right label: dodging 

### selected


 88%|████████▊ | 7/8 [00:29<00:04,  4.15s/it]

Right label: dodging 

### selected
Right label: explicit 

### selected question
Right label: dodging 

### selected
Right label: dodging 

### selected
Right label: explicit 

### selected question
Right label: dodging 

### part
Right label: implicit 

### selected question
Right label: dodging 

### selected


100%|██████████| 8/8 [00:32<00:00,  4.08s/it]

Right label: explicit 

### part of
Right label: dodging 

### selected
Right label: dodging 

### selected
Right label: implicit 

### selected question
Right label: dodging 

### selected
Right label: general 

### part of
----
0     indirect
1     indirect
2     indirect
3     indirect
4     indirect
        ...   
57    indirect
58    indirect
59    indirect
60    indirect
61    indirect
Name: clarity_label, Length: 62, dtype: object
----
----
0         indirect
1         indirect
2         indirect
3     direct reply
4         indirect
          ...     
61        indirect
62        indirect
63        indirect
64        indirect
65    direct reply
Name: clarity_label, Length: 62, dtype: object
----
['indirect', 'direct reply', 'direct non-reply']
Accuracy: 0.71
Accuracy for label indirect: 0.81
Accuracy for label direct reply: 0.76
Accuracy for label direct non-reply: 0.22

Classification Report:
                  precision    recall  f1-score   support

        indirect       0.7